In [1]:
#import libraries

import re
import pandas as pd
import os
cwd = os.getcwd()

In [3]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [4]:
#get movie folder names
movies = os.listdir('./movieSubtitles_En_Pr')

In [7]:
#Main Loop
Total_Movies = pd.DataFrame()
for mv in range(len(movies)):
    os.chdir(cwd+'//movieSubtitles_En_Pr/'+movies[mv])

    #import Reference Subtitle
    english = open( "English.srt", "r")
    english_lines = english.read()
    english.close()
    #english_lines = [i for i in english_lines if i[:-1]]
    re_en = re.findall("(\d+:\d+:\d+,\d+ --> \d+:\d+:\d+,\d+)\s+(.+)", english_lines)
    #Import Translated Subtitle
    persian = open( "Persian.srt", "r",encoding="utf8")
    persian_lines = persian.read()
    persian.close()
    re_ps = re.findall("(\d+:\d+:\d+,\d+ --> \d+:\d+:\d+,\d+)\s+(.+)", persian_lines)

    #===================Change Source ST to dictionary
    df_en = pd.DataFrame(index=range(len(re_en)), columns=['start_time','end_time','en_text'])

    for i in range(len(re_en)):
        df_en.loc[i]['start_time'] = str(re_en[i]).split(",")[0].split("'")[1]
        df_en.loc[i]['end_time'] =str(re_en[i]).split(",")[1].split("-->")[1]
        df_en.loc[i]['en_text'] = re.sub('[^A-Za-z0-9]+', ' ', remove_tags(str(re_en[i]).split(",")[3]))

    #===================Change Translated ST to dictionary
    df_ps = pd.DataFrame(index=range(len(re_ps)), columns=['start_time','end_time','pr_text'])
    for i in range(len(re_ps)):
        df_ps.loc[i]['start_time'] = str(re_ps[i]).split(",")[0].split("'")[1]
        df_ps.loc[i]['end_time'] =str(re_ps[i]).split(",")[1].split("-->")[1]

        ts3 = re.sub('[\W_]+', ' ', re.sub(r'[a-zA-Z?]', '', remove_tags(str(re_ps[i]).split(",")[3])).strip()) 
        df_ps.loc[i]['pr_text'] =  ''.join([i for i in ts3 if not i.isdigit()])
    #Combine two data frame
    df1 = pd.merge(df_en, df_ps,  how='inner', on=['start_time'])
    df2 = pd.merge(df_en, df_ps,  how='inner', on=['end_time'])
    df3 = pd.merge(df_en, df_ps,  how='inner', on=['start_time','end_time'])
    df_matched = pd.concat([df1, df2,df3])
    df_matched = df_matched.drop_duplicates(subset='pr_text', keep="first")
    df_matched.columns
    df_matched = df_matched[['en_text','pr_text']]
    df_matched.insert (0, "Movie", movies[mv])
    Total_Movies = pd.concat([Total_Movies, df_matched])


FileNotFoundError: [WinError 3] The system cannot find the path specified: './movieSubtitles_En_Pr//Batman_Begins_2005'